<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Collaborative Filtering Recommendation Algorithm Comparison

This illustrative comparison applies to collaborative filtering algorithms available in this repository such as Spark ALS, Surprise SVD and SAR. These algorithms are usable in a variety of recommendation tasks, including product or news recommendations. 

The main purpose of this notebook is not to produce comprehensive benchmarking results on multiple datasets. Rather, it is intended to illustrate on how one could evaluate different recommender algorithms using tools in this repository.

## Experimentation setup:
* Objective
  * To compare how each collaborative filtering algorithm perform in predicting ratings and recommending relevant items.
* Environment
  * The comparison is run on a [Azure Data Science Virtual Machine](https://azure.microsoft.com/en-us/services/virtual-machines/data-science-virtual-machines/). 
  * The virtual machine size is Standard NC6s_v2 (6 vcpus, 112 GB memory).
  * It should be noted that the single node DSVM is not supposed to run scalable benchmarking analysis. Either scaling up or out the computing instances is necessary to run the benchmarking in an run-time efficient way without any memory issue.
* Datasets
  * [Movielens 100K](https://grouplens.org/datasets/movielens/100k/).
  * [Movielens 1M](https://grouplens.org/datasets/movielens/1m/).
* Data split
  * The data is split into train and test sets.
  * The split ratios are 75-25 for train and test datasets.
  * The splitting is random. 
* Model training
  * A recommendation model is trained by using each of the collaborative filtering algorithms. 
  * Empirical parameter values reported [here](http://mymedialite.net/examples/datasets.html) are used in this notebook.  More exhaustive hyper parameter tuning would be required to further optimize results.
* Evaluation metrics
  * Ranking metrics:
    * Precision@k.
    * Recall@k.
    * Normalized discounted cumulative gain@k (NDCG@k).
    * Mean-average-precision (MAP). 
    * In the evaluation metrics above, k = 10. 
  * Rating metrics:
    * Root mean squared error (RMSE).
    * Mean average error (MAE).
    * R squared.
    * Explained variance.
  * Run time performance
    * Elapsed for training a model and using a model for predicting/recommending k items. 
    * The time may vary across different machines. 

## 0 Global settings

In [5]:
import sys
sys.path.append("../../")

import os
from tempfile import TemporaryDirectory

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import papermill as pm
import pyspark
import seaborn as sns

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("PySpark version: {}".format(pyspark.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Pandas version: 0.24.1
PySpark version: 2.3.1


A temporary directory is created to preserve the output notebook.

In [6]:
# Put temp results in a temp folder.
temp_dir = TemporaryDirectory()
temp_path = temp_dir.name
output_path = os.path.join(temp_path, 'output.ipynb')

In [7]:
# Global set-up parameters used to run the notebooks.
k = 10

data_sizes = ['100k', '1m']
algorithms = ['als', 'sar', 'svd', 'fast', 'lr']

notebooks = {
    'als': '../00_quick_start/als_pyspark_movielens.ipynb',
    'sar': '../00_quick_start/sar_single_node_movielens.ipynb',
    'svd': '../02_model/surprise_svd_deep_dive.ipynb',
    'fast': '../00_quick_start/fastai_recommendation.ipynb',
    'lr': '../02_model/vowpal_wabbit_deep_dive.ipynb',
}

## 1 Run notebooks to generate results

In [ ]:
# For each data size and each algorithm, a recommender is evaluated. 
df_results = pd.DataFrame()

for data_size in data_sizes:
    for algorithm in algorithms:
        print(algorithm, data_size)
        # Execute the notebook
        pm.execute_notebook(
            notebooks[algorithm],
            output_path,
            parameters = dict(TOP_K=k, MOVIELENS_DATA_SIZE=data_size)
        )
        
        # Read records from the notebook.
        nb = pm.read_notebook(output_path)
        
        # Arrange results and save them into dataframe.
        df_eval = nb.dataframe.transpose()
        df_eval = df_eval.rename(columns=df_eval.iloc[0]).drop(['name', 'type', 'filename'])
        df_eval.columns = [x.lower() for x in list(df_eval.columns)]
        
        if algorithm in ["als", "svd", "fast", "lr"]:
            df_result = pd.DataFrame(
                {
                    "Data": data_size,
                    "Algo": algorithm,
                    "K": k,
                    "MAP": df_eval['map'].item(),
                    "nDCG@k": df_eval['ndcg'].item(),
                    "Precision@k": df_eval['precision'].item(),
                    "Recall@k": df_eval['recall'].item(),
                    "RMSE": df_eval['rmse'].item(),
                    "MAE": df_eval['mae'].item(),
                    "R2": df_eval['rsquared'].item(),
                    "Explained Variance": df_eval['exp_var'].item(),
                    "Train time": df_eval['train_time'].item(),
                    "Test time": df_eval['test_time'].item()
                }, 
                index=[0]
            )
        # NOTE SAR algorithm does not predict rating scores so the rating metrics do not apply. 
        # Therefore, for SAR, the rating metrics are assigned with NAN.
        elif algorithm in ["sar"]:
            df_result = pd.DataFrame(
                {
                    "Data": data_size,
                    "Algo": algorithm,
                    "K": k,
                    "MAP": df_eval['map'].item(),
                    "nDCG@k": df_eval['ndcg'].item(),
                    "Precision@k": df_eval['precision'].item(),
                    "Recall@k": df_eval['recall'].item(),
                    "RMSE": np.nan,
                    "MAE": np.nan,
                    "R2": np.nan,
                    "Explained Variance": np.nan,
                    "Train time": df_eval['train_time'].item(),
                    "Test time": df_eval['test_time'].item()
                }, 
                index=[0]
            )
        else:
            raise ValueError("{} is not a recognized algorithm".format(algorithm))
        df_results = df_results.append(df_result, ignore_index=True)
        
df_results

als 100k



sar 100k



svd 100k



fast 100k



lr 100k



als 1m



sar 1m



svd 1m


The temporary directory is removed after completion of all runs.

## 2 Clean up

In [ ]:
# Remove the temp directory and files
temp_dir.cleanup()